# **1. Install and import necessary libraries** #

In [1]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f4d9637ac30dca60c2f4f7d069856e6a194b306545c04a072acdd5cefcbcfcd2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
# Import Spark Session
from pyspark.sql import SparkSession

In [4]:
# Create Spark Session
spark = SparkSession.builder.appName("AnalyzeDatasetwithSpark").getOrCreate()
spark.version

'3.5.0'

# **2. Read dataset** #

In [5]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Read dataset into Dataframe
df_mpg = spark.read.csv("/content/drive/MyDrive/Machine_Learning_wtih_Spark/dataset/mpg.csv", header = True, inferSchema = True)

# Print Schema of dataset
df_mpg.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [7]:
# Show few rows
df_mpg.head(5)

[Row(MPG=15.0, Cylinders=8, Engine Disp=390.0, Horsepower=190, Weight=3850, Accelerate=8.5, Year=70, Origin='American'),
 Row(MPG=21.0, Cylinders=6, Engine Disp=199.0, Horsepower=90, Weight=2648, Accelerate=15.0, Year=70, Origin='American'),
 Row(MPG=18.0, Cylinders=6, Engine Disp=199.0, Horsepower=97, Weight=2774, Accelerate=15.5, Year=70, Origin='American'),
 Row(MPG=16.0, Cylinders=8, Engine Disp=304.0, Horsepower=150, Weight=3433, Accelerate=12.0, Year=70, Origin='American'),
 Row(MPG=14.0, Cylinders=8, Engine Disp=455.0, Horsepower=225, Weight=3086, Accelerate=10.0, Year=70, Origin='American')]

# **3. Analyze data with Spark SQL** #

In [8]:
# Create temp view
df_mpg.createOrReplaceTempView("TBL_MPG")

Show full data

In [9]:
full_mpg_data = spark.sql('''
                          SELECT *
                          FROM TBL_MPG
                          ''')

full_mpg_data.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
|15.0|        8|      350.0|       165|  3693|      11.5|  70|American|
|18.0|        8|      307.0|       130|  3504|      12.0|  70|American|
|14.0|        8|      454.0|       220|  4354|       9.0|  70|American|
|15.0|        8|      400.0|       150|  3761|       9.5|  70|American|
|10.0|        8|      307.0|       200|  4376|      15.0|  70|American|
|15.0|        8|      383.0|       170|  3563|      10.0|  70|Am

Select all cars whose mileage is more than 40


In [10]:
car_mpg_mt_40 = spark.sql('''
                          SELECT *
                          FROM TBL_MPG
                          WHERE MPG > 40
                          ''')

car_mpg_mt_40.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|43.1|        4|       90.0|        48|  1985|      21.5|  78|European|
|43.4|        4|       90.0|        48|  2335|      23.7|  80|European|
|41.5|        4|       98.0|        76|  2144|      14.7|  80|European|
|44.3|        4|       90.0|        48|  2085|      21.7|  80|European|
|40.8|        4|       85.0|        65|  2110|      19.2|  80|Japanese|
|44.6|        4|       91.0|        67|  1850|      13.8|  80|Japanese|
|46.6|        4|       86.0|        65|  2110|      17.9|  80|Japanese|
|44.0|        4|       97.0|        52|  2130|      24.6|  82|European|
+----+---------+-----------+----------+------+----------+----+--------+



List all the unique Origins

In [11]:
distinct_origins = spark.sql('''
                              SELECT distinct Origin
                              FROM TBL_MPG
                            ''')
distinct_origins.show()

+--------+
|  Origin|
+--------+
|European|
|Japanese|
|American|
+--------+



Show the count of Origin Japanese cars

In [13]:
origin_japan_car = spark.sql('''
                              SELECT count(*) AS NumberOfJapanessCar
                              FROM TBL_MPG
                              WHERE Origin = 'Japanese'
                            ''')
origin_japan_car.show()

+-------------------+
|NumberOfJapanessCar|
+-------------------+
|                 79|
+-------------------+



Count the number of cars with mileage greater than 40

In [14]:
number_car_mt40 = spark.sql('''
                              SELECT count(*) AS NumberOfCarMoreThan40
                              FROM TBL_MPG
                              WHERE MPG > 0
                           ''')
number_car_mt40.show()

+---------------------+
|NumberOfCarMoreThan40|
+---------------------+
|                  392|
+---------------------+



Print the maximum MPG

In [15]:
max_mpg = spark.sql('''
                    SELECT max(MPG) AS MaxOfMPG
                    FROM TBL_MPG
                    ''')
max_mpg.show()

+--------+
|MaxOfMPG|
+--------+
|    46.6|
+--------+



# **4. Stop Spark Session** #

In [16]:
# Delete temp view
spark.catalog.dropTempView("TBL_MPG")

True

In [17]:
# Stop Spark Session
spark.stop()